In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from skimage.io import imread
import matplotlib.pyplot as plt
import pigleg_cv
import shutil

C:\Users\Jirik\miniconda3\envs\jupyterlab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'mmdet'

In [2]:
!pwd

/webapps/piglegsurgery/piglegcv


In [3]:
import torch

torch.cuda.is_available()

False

In [5]:
# print(" asdf ".strip())
if " sd ".strip():
    print("empty")

empty


In [4]:
!nvidia-smi

Wed Jun  5 22:37:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.03              Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0             19W /   65W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
torch.__version__

'1.12.0a0+8a1a93a'

In [6]:
Path("~/.cache/torch/hub/checkpoints/").expanduser().exists()

False

In [7]:
!ls ~/.cache/torch/hub/checkpoints/

ls: cannot access '/home/appuser/.cache/torch/hub/checkpoints/': No such file or directory


In [8]:
# pip install mmcv-full
!pip install pyinstrument

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 3.4 MB/s eta 0:00:00
DEPRECATION: torch-tensorrt 1.1.0a0 has a non-standard dependency specifier torch>=1.10.0+cu113<1.11.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torch-tensorrt or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


# Select video

In [9]:
is_microsurgery = False
test_first_seconds = False
n_stitches = 3
use_application_outputdir = True

patch = "*/Anasto*.mp4"
# patch = "*/2023_10_11_B-Grübbel_Mareike_Fortlaufende_Transkutannaht.mov"
# patch = "Einzel.MOV"
patch = "test_4.mp4"
n_stitches = 0
# patch = "Bad-2021_11_3_A_Brodersen___nne_Einzelknopfnaht_am_Schweinefu_"; n_stitches=0
# patch = "micro_01*.mp4"
# patch = "2023_10_11_B_Seltmann_Charlotte_fortlaufende_Transcutannaht.mp4"; n_stitches=5
# patch = "Katharina_Einzelknopfnaht"; n_stitches=3


img_pths = list(Path("../piglegsurgeryweb/media/upload/").glob(f"*/*{patch}*"))
mediafile_path = img_pths[0]


# original outputdir
if use_application_outputdir:
    outputdir = list(
        list(Path("../piglegsurgeryweb/media/").glob(f"*/*{patch}*.zip"))[
            0
        ].parent.glob("*/meta.json")
    )[0].parent
    outputdir = outputdir.absolute().resolve()
    outputdir
else:
    outputdir = Path("./del_test_video_output/") / mediafile_path.name

## Do initial processing (including tracking)


In [10]:
dcv = pigleg_cv.DoComputerVision(
    mediafile_path,
    outputdir,
    test_first_seconds=test_first_seconds,
    is_microsurgery=is_microsurgery,
)
dcv._run_tracking

dcv._make_sure_media_is_cropped()

dcv.run_image_processing()
dcv.meta["is_microsurgery"] = dcv.is_microsurgery
dcv.meta["n_stitches_by_user"] = dcv.n_stitches
dcv._run_tracking()
dcv._find_stitch_ends_in_tracks(
    n_clusters=dcv.n_stitches,
    plot_clusters=True,
    clusters_image_path=dcv.outputdir / "_stitch_clusters.jpg",
)

# dcv.run()

2024-06-05 22:37:33.577 | DEBUG    | pigleg_cv:__init__:108 - ###############
2024-06-05 22:37:33.582 | DEBUG    | pigleg_cv:__init__:109 - device=cpu
2024-06-05 22:37:33.586 | DEBUG    | pigleg_cv:__init__:110 - test_first_seconds=False
2024-06-05 22:37:33.590 | DEBUG    | pigleg_cv:__init__:111 - is_microsurgery=False
2024-06-05 22:37:33.592 | DEBUG    | pigleg_cv:__init__:142 - self.is_microsurgery=False
2024-06-05 22:37:33.596 | DEBUG    | pigleg_cv:get_parameters_for_crop_rotate_rescale:464 - device=cpu
2024-06-05 22:37:33.714 | DEBUG    | run_perpendicular:get_frame_to_process:180 - last_frame=152
2024-06-05 22:37:33.715 | DEBUG    | run_perpendicular:get_frame_to_process:181 - reference_frame_position_from_end=0
2024-06-05 22:37:33.716 | DEBUG    | run_perpendicular:get_frame_to_process:182 - step=5
/home/appuser/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually re

Fusing layers... 
IDetect.fuse


/opt/conda/lib/python3.8/site-packages/torch/functional.py:598: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2323.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-06-05 22:38:23.419 | DEBUG    | run_qr:bbox_info_extraction_from_frame:240 - qr code text = 'Resolution 30 mm'
2024-06-05 22:38:23.431 | DEBUG    | run_qr:bbox_info_extraction_from_frame:240 - qr code text = 'Scissors 30 mm'
2024-06-05 22:38:23.436 | DEBUG    | run_qr:bbox_info_extraction_from_frame:340 - qr_data={'is_microsurgery': False, 'pix_size_method': 'weighted_average', 'is_detected': True, 'box': [[410, 287], [532, 286], [535, 164], [411, 164]], 'pix_size': 0.0002446659365537673, 'incision_bboxes': [[774.2468872070312, 706.2264404296875, 1064.2303466796875, 784.998291015625, 0.9780232310295105]], 'qr_size': 0.03, 'size_by_scene': 0.00015625, 'text'

load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/incision_detection_models/220326_234659_mmdet.pth


2024-06-05 22:38:46.641 | DEBUG    | incision_detection_mmdet:save_incision_cropped_images:291 - number of detected incisions = 1
2024-06-05 22:38:46.668 | DEBUG    | pigleg_cv:get_parameters_for_crop_rotate_rescale:483 - qr_data={'is_microsurgery': False, 'pix_size_method': 'weighted_average', 'is_detected': True, 'box': [[410, 287], [532, 286], [535, 164], [411, 164]], 'pix_size': 0.0002446659365537673, 'incision_bboxes': [[774.2468872070312, 706.2264404296875, 1064.2303466796875, 784.998291015625, 0.9780232310295105]], 'qr_size': 0.03, 'size_by_scene': 0.00015625, 'text': 'Resolution 30 mm', 'pix_size_single_frame_detector_m': 0.00023340120243941213, 'bbox_scene_area': [237.48182678222656, 1.596093773841858, 1683.50341796875, 1080.0, 0.6937737464904785], 'bbox_micro_calibration': [], 'qr_scissors_frame_detected': True, 'qr_bboxes_SID': [[404.9392395019531, 158.84629821777344, 542.061279296875, 293.9641418457031, 0.9962145686149597], [1111.7254638671875, 165.35377502441406, 1253.7729

...

load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/tracker_model_bytetrack/epoch.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_backbone_stage1_1_main_conv_bn_running_mean, ema_detector_backbone_stage1

2024-06-05 22:41:19,822 - mmtrack - INFO - initialize CSPDarknet with init_cfg {'type': 'Kaiming', 'layer': 'Conv2d', 'a': 2.23606797749979, 'distribution': 'uniform', 'mode': 'fan_in', 'nonlinearity': 'leaky_relu'}
2024-06-05 22:41:20,033 - mmtrack - INFO - initialize YOLOXPAFPN with init_cfg {'type': 'Kaiming', 'layer': 'Conv2d', 'a': 2.23606797749979, 'distribution': 'uniform', 'mode': 'fan_in', 'nonlinearity': 'leaky_relu'}
2024-06-05 22:41:20,162 - mmtrack - INFO - initialize YOLOXHead with init_cfg {'type': 'Kaiming', 'layer': 'Conv2d', 'a': 2.23606797749979, 'distribution': 'uniform', 'mode': 'fan_in', 'nonlinearity': 'leaky_relu'}


load checkpoint from local path: /webapps/piglegsurgery/piglegcv/resources/tracker_model_bytetrack_hands_tools/epoch.pth
The model and loaded state dict do not match exactly

size mismatch for detector.bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([5, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([3, 256, 1, 1]).
size mismatch for detector.bbox_head.multi_level_conv_cls.0.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).
size mismatch for detector.bbox_head.multi_level_conv_cls.1.weight: copying a param with shape torch.Size([5, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([3, 256, 1, 1]).
size mismatch for detector.bbox_head.multi_level_conv_cls.1.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).
size mismatch for detector.bbox_head.multi_level_conv_cls.2.weight: copying a param w

2024-06-05 22:41:56.155 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:157 - Tracking on frame 0, 0.00% (0.13s elapsed, 1000.00s remaining)
2024-06-05 22:44:41.087 | DEBUG    | run_tracker_bytetrack:main_tracker_bytetrack:157 - Tracking on frame 50, 52.63% (165.05s elapsed, 148.54s remaining)
2024-06-05 22:47:13.933 | DEBUG    | pigleg_cv:find_stitch_ends_in_tracks:848 - find_stitch_end, n_clusters=0, outputdir=PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media/SA_20231213-082852_yogjl47cusic/SA_20231213-082852')
2024-06-05 22:47:13.956 | DEBUG    | pigleg_cv:_get_metadata:786 - incision_bboxes=[[264.8380432128906, 341.15032958984375, 417.4794921875, 394.9026184082031, 0.9969009160995483]]
2024-06-05 22:47:13.960 | DEBUG    | pigleg_cv:_get_metadata:788 - find stitch end, pix_size=0.0004876054523338215, fps=15
2024-06-05 22:47:13.974 | DEBUG    | pigleg_cv:_get_X_px_fr_more_tools:732 - X_px_fr_list=[array([[     269.16,      374.19,          49],
       [     269.16,  

[0, 94]

In [ ]:
## deterministic profiles
# from cProfile import Profile
# from pstats import SortKey, Stats
# with Profile() as profile:
#     dcv._make_report(cut_frames=dcv.meta["stitch_split_frames"])(
#         Stats(profile)
#         .strip_dirs()
#         .sort_stats(SortKey.CALLS)
#         .print_stats()
#     )

In [ ]:
from pyinstrument import Profiler

with Profiler(interval=0.02) as profiler:
    dcv._make_report(cut_frames=dcv.meta["stitch_split_frames"])

In [35]:
profiler.print()


  _     ._   __/__   _ _  _  _ _/_   Recorded: 00:19:17  Samples:  355
 /_//_/// /_\ / //_// / //_'/ //     Duration: 10.830    CPU time: 9.212
/   _/                      v4.6.1

Program: /opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/appuser/.local/share/jupyter/runtime/kernel-f704adcb-7f3f-42c6-9361-10e73b240a0b.json

10.810 <cell line: 2>  ../../../tmp/ipykernel_505/356217478.py:2
└─ 10.810 DoComputerVision._make_report  pigleg_cv.py:529
   └─ 10.810 main_report  run_report.py:906
      ├─ 2.925 plot3  run_report.py:500
      │  └─ 2.905 Figure.savefig  matplotlib/figure.py:2923
      │        [38 frames hidden]  matplotlib
      ├─ 2.485 system  <built-in>
      ├─ 1.866 create_pdf_report_for_one_tool  run_report.py:219
      │  ├─ 1.255 savefig  matplotlib/pyplot.py:976
      │  │     [34 frames hidden]  matplotlib, <built-in>, PIL
      │  ├─ 0.222 Figure.tight_layout  matplotlib/figure.py:3189
      │  │     [6 frames hidden]  matplotlib
      │  └─ 0.12

In [29]:
# with skimage.transform.resize
profiler.print()


  _     ._   __/__   _ _  _  _ _/_   Recorded: 00:03:00  Samples:  701
 /_//_/// /_\ / //_// / //_'/ //     Duration: 16.762    CPU time: 26.226
/   _/                      v4.6.1

Program: /opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/appuser/.local/share/jupyter/runtime/kernel-f704adcb-7f3f-42c6-9361-10e73b240a0b.json

16.754 <cell line: 2>  ../../../tmp/ipykernel_505/356217478.py:2
└─ 16.754 DoComputerVision._make_report  pigleg_cv.py:529
   └─ 16.754 main_report  run_report.py:906
      ├─ 6.859 resize  skimage/transform/_warps.py:19
      │     [20 frames hidden]  skimage, <__array_function__ internal...
      │        5.271 warp  skimage/transform/_warps.py:684
      │        ├─ 3.970 [self]  skimage/transform/_warps.py
      ├─ 4.709 plot3  run_report.py:500
      │  └─ 4.709 Figure.savefig  matplotlib/figure.py:2923
      │        [44 frames hidden]  matplotlib
      ├─ 1.907 create_pdf_report_for_one_tool  run_report.py:219
      │  ├─ 1.359 savefig  m

In [30]:
profiler.open_in_browser()

'/tmp/tmpwg8lzr2x.html'

In [12]:
# report_data = dcv.make_report()